In [32]:
import pandas as pd
import datetime
import os

In [17]:
# def data_process(s_1, s_2):
#     file_comment = pd.read_csv(s_1, encoding='ISO-8859-1')
#     file_submission = pd.read_csv(s_2, encoding='ISO-8859-1')
    
#     if len(file_comment) == 0 and len(file_submission) == 0:
#         return
    
#     selected_cols = ['author', 'subreddit', 'subreddit_id']
#     if 'utc_datetime_str' in file_comment.columns:
#         selected_cols.append('utc_datetime_str')

#     file_comment = file_comment[selected_cols].sort_values('author')
#     file_submission = file_submission[selected_cols].sort_values('author')

#     s = file_comment.loc[0, 'subreddit'] if len(file_comment.index) != 0 else file_submission.loc[0, 'subreddit']
#     s_id = file_comment.loc[0, 'subreddit_id'] if len(file_comment.index) != 0 else file_submission.loc[0, 'subreddit_id']
    
#     if 'utc_datetime_str' in file_comment.columns:
#         x = datetime.datetime(2020, 11, 4)

#         file_comment['utc_datetime_str'] = pd.to_datetime(file_comment['utc_datetime_str'], errors='coerce')
#         file_submission['utc_datetime_str'] = pd.to_datetime(file_submission['utc_datetime_str'], errors='coerce')
    
#         file_comment = file_comment[(file_comment['author'] != '[deleted]') & 
#                                     (file_comment['utc_datetime_str'] < x) & 
#                                     (file_comment['utc_datetime_str'].notna())]
#         file_submission = file_submission[(file_submission['author'] != '[deleted]') & 
#                                           (file_submission['utc_datetime_str'] < x) & 
#                                           (file_submission['utc_datetime_str'].notna())]
#     else:
#         file_comment = file_comment[file_comment['author'] != '[deleted]']
#         file_submission = file_submission[file_submission['author'] != '[deleted]']

#     file_new = pd.concat([file_comment, file_submission])
    
#     file_count = file_new.groupby('author').count().reset_index()

#     file_output = file_count[['author', 'subreddit']]
#     file_output = file_output.rename(columns = {'subreddit':'n_posts'})
#     file_output['subreddit'] = s
#     file_output['subreddit_id'] = s_id
    
#     name = '_'.join(s_1.split('_')[:-1]) + '_posts.csv'
#     file_output.to_csv(name, index = False)

In [33]:
def process_two(s_1, s_2):
    file_comment = pd.read_csv(s_1, encoding='ISO-8859-1')
    file_submission = pd.read_csv(s_2, encoding='ISO-8859-1')
    file_title_comment = s_1.split('_')[0].lower()
    
    if len(file_comment) == 0 and len(file_submission) == 0:
        return
    
    selected_cols = ['author', 'subreddit', 'subreddit_id']
    if 'utc_datetime_str' in file_comment.columns:
        selected_cols.append('utc_datetime_str')

    file_comment = file_comment[selected_cols].sort_values('author', ignore_index = True)
    file_submission = file_submission[selected_cols].sort_values('author', ignore_index = True)
    
    if 'utc_datetime_str' in file_comment.columns:
        x = datetime.datetime(2020, 11, 4)

        file_comment['utc_datetime_str'] = pd.to_datetime(file_comment['utc_datetime_str'], errors='coerce')
        file_submission['utc_datetime_str'] = pd.to_datetime(file_submission['utc_datetime_str'], errors='coerce')
    
        file_comment = file_comment[(file_comment['author'] != '[deleted]') & 
                                    (file_comment['utc_datetime_str'] < x) & 
                                    (file_comment['utc_datetime_str'].notna())]
        file_comment['subreddit'] = file_comment['subreddit'].apply(str.lower)
        file_comment = file_comment[file_comment['subreddit'] == file_title_comment]
        
        file_submission = file_submission[(file_submission['author'] != '[deleted]') & 
                                          (file_submission['utc_datetime_str'] < x) & 
                                          (file_submission['utc_datetime_str'].notna())]
        file_submission['subreddit'] = file_submission['subreddit'].apply(str.lower)
        file_submission = file_submission[file_submission['subreddit'] == file_title_comment]

    else:
        file_comment = file_comment[file_comment['author'] != '[deleted]']
        file_comment['subreddit'] = file_comment['subreddit'].apply(str.lower)
        file_comment = file_comment[file_comment['subreddit'] == file_title_comment]
        
        file_submission = file_submission[file_submission['author'] != '[deleted]']
        file_submission['subreddit'] = file_submission['subreddit'].apply(str.lower)
        file_submission = file_submission[file_submission['subreddit'] == file_title_comment]
        
    file_new = pd.concat([file_comment, file_submission])
    file_count = file_new.groupby('author').count().reset_index()
    
    file_output = file_count[['author', 'subreddit']]
    file_output = file_output.rename(columns = {'subreddit':'n_posts'})
    
    most_common_subreddit = file_new.groupby('author')['subreddit'].agg(pd.Series.mode).reset_index()
    most_common_subreddit_id = file_new.groupby('author')['subreddit_id'].agg(pd.Series.mode).reset_index()

    file_output = pd.merge(file_output, most_common_subreddit, on='author')
    file_output = pd.merge(file_output, most_common_subreddit_id, on='author')
    name = '_'.join(s_1.split('_')[:-1]) + '_posts.csv'
    file_output.to_csv(name, index = False)

In [18]:
#Get all csv files names
current_directory = os.getcwd()

files = []
for i in os.listdir(current_directory):
    if i.endswith('.csv'):
        files.append(i)
        
file_names = pd.DataFrame({'file_name':files})
file_names

,file_name
0,Nacogdoches_comments.csv
1,Nacogdoches_submissions.csv
2,nampa_comments.csv
3,nampa_submissions.csv
4,Nantucket_comments.csv
...,...
315,queens_submissions.csv
316,quinceyma_comments.csv
317,quinceyma_submissions.csv
318,quincy_comments.csv


In [19]:
#Process all csv files
even = 0
odd = 1
count = 1
while even < len(file_names) and odd < len(file_names):
    s_1 = file_names.loc[even, 'file_name']
    s_2 = file_names.loc[odd, 'file_name']
    
    #Check if one area has both comments and submissions
    #If not, skip
    two_files = True
    if '_'.join(s_1.split('_')[:-1]) == '_'.join(s_2.split('_')[:-1]):
        data_process(s_1, s_2)
    else:
        two_files = False
        even += 1
        odd += 1
    
    if two_files == True:
        even += 2
        odd += 2
    
    print(count)
    count += 1

1
2
3
4
5
6
7
8
9
10
11


C:\Users\chris\AppData\Local\Temp\ipykernel_12484\2513941790.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  file_comment = pd.read_csv(s_1, encoding='ISO-8859-1')
C:\Users\chris\AppData\Local\Temp\ipykernel_12484\2513941790.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  file_submission = pd.read_csv(s_2, encoding='ISO-8859-1')


12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


C:\Users\chris\AppData\Local\Temp\ipykernel_12484\2513941790.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  file_comment = pd.read_csv(s_1, encoding='ISO-8859-1')


51
52
53
54
55


...

103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140


C:\Users\chris\AppData\Local\Temp\ipykernel_12484\2513941790.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  file_comment = pd.read_csv(s_1, encoding='ISO-8859-1')
C:\Users\chris\AppData\Local\Temp\ipykernel_12484\2513941790.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  file_submission = pd.read_csv(s_2, encoding='ISO-8859-1')


141
142
143
144
145
146
147
148
149
150


C:\Users\chris\AppData\Local\Temp\ipykernel_12484\2513941790.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  file_comment = pd.read_csv(s_1, encoding='ISO-8859-1')
C:\Users\chris\AppData\Local\Temp\ipykernel_12484\2513941790.py:3: DtypeWarning: Columns (1,2,8) have mixed types. Specify dtype option on import or set low_memory=False.
  file_submission = pd.read_csv(s_2, encoding='ISO-8859-1')


151
152
153
154
155
156
157
158
159
160


In [20]:
print(s_1)
print(s_2)

quincy_comments.csv
quincy_submissions.csv


In [21]:
#Remove submission and comments files
for i in os.listdir(current_directory):
    if i.endswith('comments.csv') or i.endswith('submissions.csv'):
        os.remove(i)

In [35]:
import glob

files = glob.glob('*.csv')

comments = sorted([f for f in files if '_comment' in f])
submissions = sorted([f for f in files if '_submission' in f])

paired_files = list(zip(comments, submissions))

[('Bagdad_comments.csv', 'Bagdad_submissions.csv'), ('BakersfieldCA_comments.csv', 'BakersfieldCA_submissions.csv'), ('Bakersfield_comments.csv', 'Bakersfield_submissions.csv'), ('BaldwinCity_comments.csv', 'BaldwinCity_submissions.csv'), ('Bardstown_comments.csv', 'Bardstown_submissions.csv'), ('BarrowAK_comments.csv', 'BarrowAK_submissions.csv'), ('BartlettIllinois_comments.csv', 'BartlettIllinois_submissions.csv'), ('Batavia_comments.csv', 'Batavia_submissions.csv'), ('BatesvilleUnited_comments.csv', 'BatesvilleUnited_submissions.csv'), ('Batesville_comments.csv', 'Batesville_submissions.csv'), ('Bathgate_comments.csv', 'Bathgate_submissions.csv'), ('BayArea_comments.csv', 'BayArea_submissions.csv'), ('BayCityMichigan_comments.csv', 'BayCityMichigan_submissions.csv'), ('BayonneNJ_comments.csv', 'BayonneNJ_submissions.csv'), ('BeaumontCa_comments.csv', 'BeaumontCa_submissions.csv'), ('Beaumont_comments.csv', 'Beaumont_submissions.csv'), ('BeaverCounty_comments.csv', 'BeaverCounty_sub

In [36]:
for paired_file in paired_files: 
    s_1, s_2 = paired_file
    process_two(s_1, s_2)
    os.remove(s_1)
    os.remove(s_2)
    print(s_1, s_2)

Bagdad_comments.csv Bagdad_submissions.csv
BakersfieldCA_comments.csv BakersfieldCA_submissions.csv
Bakersfield_comments.csv Bakersfield_submissions.csv
BaldwinCity_comments.csv BaldwinCity_submissions.csv
Bardstown_comments.csv Bardstown_submissions.csv
BarrowAK_comments.csv BarrowAK_submissions.csv
BartlettIllinois_comments.csv BartlettIllinois_submissions.csv
Batavia_comments.csv Batavia_submissions.csv
BatesvilleUnited_comments.csv BatesvilleUnited_submissions.csv
Batesville_comments.csv Batesville_submissions.csv
Bathgate_comments.csv Bathgate_submissions.csv


C:\Users\chris\AppData\Local\Temp\ipykernel_12484\1835853314.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  file_comment = pd.read_csv(s_1, encoding='ISO-8859-1')
C:\Users\chris\AppData\Local\Temp\ipykernel_12484\1835853314.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  file_submission = pd.read_csv(s_2, encoding='ISO-8859-1')


BayArea_comments.csv BayArea_submissions.csv
BayCityMichigan_comments.csv BayCityMichigan_submissions.csv
BayonneNJ_comments.csv BayonneNJ_submissions.csv
BeaumontCa_comments.csv BeaumontCa_submissions.csv
Beaumont_comments.csv Beaumont_submissions.csv
BeaverCounty_comments.csv BeaverCounty_submissions.csv
BeaverPA_comments.csv BeaverPA_submissions.csv
BeckleyOnReddit_comments.csv BeckleyOnReddit_submissions.csv
BelAirMD_comments.csv BelAirMD_submissions.csv
BellCountyTX_comments.csv BellCountyTX_submissions.csv
BellevueKY_comments.csv BellevueKY_submissions.csv
Belmar_comments.csv Belmar_submissions.csv
Beltsville_comments.csv Beltsville_submissions.csv
Bethalto_comments.csv Bethalto_submissions.csv
BethelAK_comments.csv BethelAK_submissions.csv
BeverlyBeach_comments.csv BeverlyBeach_submissions.csv
BigBear_comments.csv BigBear_submissions.csv
BigBendTX_comments.csv BigBendTX_submissions.csv
Bishop_comments.csv Bishop_submissions.csv
BismarckMandan_comments.csv BismarckMandan_submissi

C:\Users\chris\AppData\Local\Temp\ipykernel_12484\1835853314.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  file_comment = pd.read_csv(s_1, encoding='ISO-8859-1')
C:\Users\chris\AppData\Local\Temp\ipykernel_12484\1835853314.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  file_submission = pd.read_csv(s_2, encoding='ISO-8859-1')


baltimore_comments.csv baltimore_submissions.csv
bangorme_comments.csv bangorme_submissions.csv
barre_comments.csv barre_submissions.csv
barringtonil_comments.csv barringtonil_submissions.csv
barrowcounty_comments.csv barrowcounty_submissions.csv
bartlett_comments.csv bartlett_submissions.csv
bastroptx_comments.csv bastroptx_submissions.csv


...

brooklyn_comments.csv brooklyn_submissions.csv
broward_comments.csv broward_submissions.csv
brysoncity_comments.csv brysoncity_submissions.csv
btv_comments.csv btv_submissions.csv
buchanan_comments.csv buchanan_submissions.csv
buckscountypa_comments.csv buckscountypa_submissions.csv


C:\Users\chris\AppData\Local\Temp\ipykernel_12484\1835853314.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  file_comment = pd.read_csv(s_1, encoding='ISO-8859-1')


buffalo_comments.csv buffalo_submissions.csv
buffalominnesota_comments.csv buffalominnesota_submissions.csv
buffaloriver_comments.csv buffaloriver_submissions.csv
bullcity_comments.csv bullcity_submissions.csv
buncombe_comments.csv buncombe_submissions.csv
burbank_comments.csv burbank_submissions.csv
burien_comments.csv burien_submissions.csv
burleson_comments.csv burleson_submissions.csv
burlingame_comments.csv burlingame_submissions.csv


C:\Users\chris\AppData\Local\Temp\ipykernel_12484\1835853314.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  file_comment = pd.read_csv(s_1, encoding='ISO-8859-1')


burlington_comments.csv burlington_submissions.csv
burlingtonma_comments.csv burlingtonma_submissions.csv
burlingtonvt_comments.csv burlingtonvt_submissions.csv
burnsvillemn_comments.csv burnsvillemn_submissions.csv
bushwick_comments.csv bushwick_submissions.csv
butte_comments.csv butte_submissions.csv
butus_comments.csv butus_submissions.csv
bville_comments.csv bville_submissions.csv


In [ ]:
for i in file_names['file_name']:
    os.remove(i)